# El Lab (DAA Project)

En un laboratorio de la BIOFAM quieren modificar genéticamente una gallina para que sea más resistente a las enfermedades. Para esto deben de cambiar la secuencia de aminoácidos presentes en su genoma. La secuencia que quieren reproducir es la de los cocodrilos, que se sabe que son animales que han existido por millones de años. Entonces, dado que una secuencia de aminoácidos se puede codificar como una secuencia de caracteres se tiene:  

- $T$ es la secuencia de aminoácidos de los cocodrilos (tamaño $m$)  
- $S$ es la secuencia de aminoácidos de la gallina original (tamaño $n$)
- $A$ es la secuencia de aminoácidos de la galllina a crear (inicialmente vacía)

Para lograr que la gallina modificada adquiera la resistencia de un cocodrilo se sabe que su secuencia de aminoácidos $A$, debe contener como prefijo a $T$. Como en la BIOFAM hubo recortes de presupuesto solo cuentan con una máquina que puede hacer dos modificaciones genéticas:  
1. Extrae y elimina el primer aminoácido de $S$ y lo coloca al principio de $A$  
2. Extrae y elimina el primer aminoácido de $S$ y lo coloca al final de $A$  

Encuentre la **cantidad de secuencias de operaciones posibles** tales que la gallina genéticamente modificada sea resistente a las enfermedades.  

Aclaraciones del problema:
+ Se debe modificar la gallina ($A$) con todos los caracteres de $S$, no basta con solo tener el prefijo, se debe llegar al final.

## Soluciones
Para una primera solución se realizó un algoritmo en el que se hace un backtrack por cada una de las operaciones posibles para crear la cadena $A$ comprobando finalmente si contiene a la cadena $T$ como prefijo dando como resultado una secuencia posible de operaciones.

In [36]:
def count_backtrack(T:str, S:str):
    if 0 < len(T) and len(T) <= len(S):
        def backtrack(T:str, S:str, index:int, A:str):
            if len(S) == index:
                if T == A[0:len(T)]:return 1
                return 0
            char = S[index]
            cant = backtrack(T, S, index+1, char+A)
            cant += backtrack(T, S, index+1, A+char)
            return cant
        return backtrack(T, S, 0, "")
    return 0

Probando el algoritmo con distintos casos se obtiene como resultado:

In [37]:
print(count_backtrack('ACAC','ACACBCA'))
print(count_backtrack('ABAB','BABA'))
print(count_backtrack('ACABD','ACABDBCACA'))
print(count_backtrack('DBACA', 'ACABD'))
print(count_backtrack('AABBAA','BABAAA'))
print(count_backtrack('AABBAA','CHAABBAA'))
print(count_backtrack('ACAB','BDACACBA'))

12
4
20
4
12
4
8


Para el ejemplo donde $T$ (prefijo) es "ABAB" y la cadena $S$ es BABA se tiene:  
1. B $\rightarrow$ BA $\rightarrow$ BAB $\rightarrow$ BABA  (poniendo B al principio y al final)  
2. B $\rightarrow$ AB $\rightarrow$ BAB $\rightarrow$ BABA  (poniendo B al principio y al final)  
 
*Resultado*: 4

Se puede apreciar que, aunque el algoritmo resuelve el problema presenta una gran complejidad temporal, la ventaja radica en que para próximos algoritmos que tengan menor complejidad temporal se pueda comprobar si esas soluciones planteadas van por el camino correcto. Se conoce que $|T| = m$ y $|S| = n$, por tanto:

\begin{equation*}
    T(n,m) = 2 T(n-1) + 1
\end{equation*} 

donde por Teorema Maestro pertence a $\theta(2^n)$

Se puede observar que la primera letra colocándola al principio o al final genera exactamente la misma ramificación, por lo que sería mejor evitar hacer esta segunda ramificación innecesariamente, para ello:

In [38]:
def count_backtrack_modified(T:str, S:str):
    if 0 < len(T) and len(T) <= len(S):
        def backtrack(T:str, S:str, index:int, A:str):
            if len(S) == index:
                if T == A[0:len(T)]:return 1
                return 0
            char = S[index]
            cant = backtrack(T, S, index+1, char+A)
            cant += backtrack(T, S, index+1, A+char)
            return cant
        return 2*backtrack(T, S, 1, S[0])
    return 0

Probando este algoritmo obtenemos los mismmos resultados, con una ligera mejora en eficiencia:

In [39]:
print(count_backtrack_modified('ACAC','ACACBCA'))
print(count_backtrack_modified('ACAC','CACA'))
print(count_backtrack_modified('ACABD','ACABDBCACA'))
print(count_backtrack_modified('DBACA', 'ACABD'))
print(count_backtrack_modified('AABBAA','BABAAA'))
print(count_backtrack_modified('AABBAA','CHAABBAA'))
print(count_backtrack_modified('ACAB','BDACACBA'))

12
4
20
4
12
4
8


Como propuesta para una mejor solución a lo anterior, se realizó un memoize ya que en algunos llamados se hacían cálculos sobre subcadenas que ya se habían realizado con anterioridad, por lo que sería mejor guardar por subsecuencias de la cadena formada $A$, la subcadena restante de $S$, así como la cantidad de subsecuencias de operaciones posibles realizadas.

In [40]:
def count_backtrack_memoize(T:str, S:str):
    if 0 < len(T) and len(T) <= len(S):
        memoize = {}
        def backtrack(T:str, S:str, index:int, A:str):
            if len(S) == index:
                if T == A[0:len(T)]:return 1
                return 0
            char = S[index]
            dict_str = S[index+1:len(S)]
            word_form = char + A
            if dict_str in memoize and word_form in memoize[dict_str]: 
                cant = memoize[dict_str][word_form]
            else: 
                cant = backtrack(T, S, index+1, word_form)
                memoize[dict_str] = {word_form : cant}
            word_form = A + char
            if dict_str in memoize and word_form in memoize[dict_str]: 
                cant += memoize[dict_str][word_form]
            else:
                cant2 = backtrack(T, S, index+1, word_form)
                memoize[dict_str] = {word_form : cant2}
                cant += cant2
            return cant
        return 2*backtrack(T, S, 1, S[0])
    return 0

Probando el algoritmo se puede comprobar que en efecto se obtienen los mismos resultados:

In [41]:
print(count_backtrack_memoize('ACAC','ACACBCA'))
print(count_backtrack_memoize('ABAB','BABA'))
print(count_backtrack_memoize('ACABD','ACABDBCACA'))
print(count_backtrack_memoize('DBACA', 'ACABD'))
print(count_backtrack_memoize('AABBAA','BABAAA'))
print(count_backtrack_memoize('AABBAA','CHAABBAA'))
print(count_backtrack_memoize('ACAB','BDACACBA'))

12
4
20
4
12
4
8


Aunque este algoritmo constituye una mejora significativa, sigue presentando una complejidad temporal excesiva, ya que en el peor de los casos no se repiten patrones (subcadenas) y por tanto sigue perteneciendo a $O(2^n)$ con una complejidad espacial superior. La ventaja radica en que para próximos algoritmos que tengan mejor complejidad temporal se pueda comprobar si esas soluciones planteadas van por el camino correcto.

Fue necesario encontrar otras vías para solucionar el ejercicio. Partiremos de los posibles casos en que puede aparecer la cadena $T$ (el prefijo), en la cadena creada (genéticamente modificada) $A$.  

Si se comienza a leer la cadena que se desea modificar desde atrás hacia adelante se puede observar que:
+ El último elemento podría ser colocado hacia adelante si dicho caracter es igual al primer elemento del prefijo dado que luego de él no se va a poder colocar ningún otro elemento en la cadena.
    + A partir de este momento, el problema se reduce a calcular la cantidad de secuencias de operaciones en la que se puede formar un genoma modificado desde el inicio de la cadena hasta el caracter anterior a este elemento y el prefijo desde el segundo elemento hasta el final:  
    $method(p_1 p_2 p_3 ... p_i~~~~y~~~~a_1 a_2 ... a_{j-1} p_1)$ $\rightarrow$ $method(p_2 p_3 ... p_i~~~~y~~~~a_1 a_2 ... a_{j-1})$
+ El último elemento igualmente siempre se puede poner detrás
    + Si no pertence al prefijo, el problema se reduce a generar una secuencia de operaciones tal que se obtenga una cadena a partir del resto de esta (sin dicho elemento) teniendo en cuenta todo el prefijo:  
    $method(p_1 p_2 p_3 ... p_i~~~~y~~~~a_1 a_2 a_3... a_j)$ $\rightarrow$ $method(p_1 p_2 p_3 ... p_i~~~~y~~~~a_1 a_2 ... a_{j-1})$
+ De igual forma este caracter se puede considerar poner detrás, pero si pertence al prefijo entonces debe ser el último elemento del mismo ya que es el último que se coloca, siendo el tamaño del prefijo igual al de la cadena
    + Este problema se reduce a generar con el resto de la cadena el resto del prefijo:  
    $method(p_1 p_2 p_3 ... p_i~~~~y~~~~a_1 a_2 a_3... p_i)$ $\rightarrow$ $method(p_1 p_2 p_3 ... p_{i-1}~~~~y~~~~a_1 a_2 ... a_{j-1})$

In [42]:
def count_naive_recursive(T:str, S:str):
    if 0 < len(T) and len(T) <= len(S):
        def naive_recursive(S,T):
            if len(S) < len(T): return 0
            if len(S)==1: 
                if len(T)==1 and T[0]==S[0]: return 2
                return 0
            if len(T)==0 : return 0
            output = naive_recursive(S[0:len(S)-1],T)
            if S[-1]==T[0]: 
                if len(T)==1: output += 2**(len(S)-1)
                else: output += naive_recursive(S[0:len(S)-1],T[1:])
            if len(S)==len(T) and S[-1]==T[-1]: output += naive_recursive(S[0:len(S)-1],T[0:len(T)-1])
            return output
        return naive_recursive(S,T)
    return 0

Probando el algoritmo se puede comprobar que en efecto se obtienen los mismos resultados:

In [43]:
print(count_naive_recursive('ACAC','ACACBCA'))
print(count_naive_recursive('ABAB','BABA'))
print(count_naive_recursive('ACABD','ACABDBCACA'))
print(count_naive_recursive('DBACA', 'ACABD'))
print(count_naive_recursive('AABBAA','BABAAA'))
print(count_naive_recursive('AABBAA','CHAABBAA'))
print(count_naive_recursive('ACAB','BDACACBA'))

12
4
20
4
12
4
8


Para este algoritmo se hizo uso de dp guardando los resultados de los cálculos anteriores siguiendo el flujo del algoritmo anterior con sus 3 índices.  
Por lo que este algoritmo pertenece a $O(n^2*m)$

In [44]:
def count_dynamic(T:str, S:str):
    if len(T) > len(S): return 0
    dp=[ [ [0 for i in range(len(S))] for i in range(len(T))] for i in range(len(T)) ]
    i=len(T)-1
    while i>=0:
        for j in range(i,len(T)):
            for k in range(len(S)):
                m=j-i
                if m > k:
                    dp[i][j][k]=0
                    continue
                if k==0:
                    if S[0] == T[i]: dp[i][j][k]=2
                    else: dp[i][j][k]=0                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                   
                    continue
                if k >0 : dp[i][j][k]=dp[i][j][k-1]
                if S[k] == T[i]:
                    if m==0: dp[i][j][k]+=2**k
                    elif i<len(T)-1 and k>0: dp[i][j][k]+=dp[i+1][j][k-1]
                if k==m and S[k] == T[j] and j>0 and k>0: dp[i][j][k]+=dp[i][j-1][k-1]
        i-=1
    return dp[0][len(T)-1][len(S)-1]

Probando el algoritmo se puede comprobar que en efecto se obtienen los mismos resultados:

In [45]:
print(count_dynamic('ACAC','ACACBCA'))
print(count_dynamic('ABAB','BABA'))
print(count_dynamic('ACABD','ACABDBCACA'))
print(count_dynamic('DBACA', 'ACABD'))
print(count_dynamic('AABBAA','BABAAA'))
print(count_dynamic('AABBAA','CHAABBAA'))
print(count_dynamic('ACAB','BDACACBA'))

12
4
20
4
12
4
8


Explicaciones del nuevo algoritmo!!!

In [83]:
def solve_dp(T,S):
    n = len(S)
    m = len(T)
        
    if m > n:
        return 0
        
    dp = [[0 for j in range(n)] for i in range(n)]
    
    for i in range(n):
        if i >= m or T[i] == S[0]:
            dp[i][i] = 2
    
    for k in range(1, n):    
        c = S[k]
        
        i = 0        
        for j in range(k, n):
            if i >= m or c == T[i]:
                dp[i][j] += dp[i+1][j]
            if j >= m or c == T[j]:
                dp[i][j] += dp[i][j-1]            
            i += 1            
    dp
    return dp[0][-1]

Probando el algoritmo se puede comprobar que en efecto se obtienen los mismos resultados:

In [22]:
# Probando el codigo

En las siguientes líneas de código se realizó un generador de cadenas que juegan el rol de prefijo (cocodrilo) y genoma a modificar (gallina) haciendo particiones en el prefijo, insertando un caracter random en alguna partición e invirtiendo la secuencia. Se comprueban los resultados entre los algoritmos.

In [96]:
import random
VOC = ["C","T","G","A","U","R","D"]

def generate_prefix(a=10,b=15):
    length = random.randint(a, b)
    prefix = ""
    while length > 0:
        letter = random.randint(0, len(VOC)-1)
        pos = random.randint(0,1)
        if pos:
            prefix += VOC[letter]
        else:
            prefix = VOC[letter] + prefix
        length-=1
    return prefix

def generate_secuence(prefix:str):
    secuence = ""
    while len(secuence) < 40:
        letter = random.randint(0, len(VOC)-1)
        partition = list(prefix.partition(random.choice(prefix)))
        pos = random.randint(0,len(partition))
        partition.insert(pos, VOC[letter])
        secuence += "".join(reversed(partition))
        secuence
    return secuence
   
TEST = 1
while TEST <= 100:
    print(f'TEST NO. {TEST}')
    prefix = generate_prefix()
    secuence = generate_secuence(prefix)
    print(f'PREFIX: {prefix} \nSECUENCE: {secuence}')
    #c_back = count_backtrack_memoize(prefix,secuence)
    #print(f'Naive Recursive: {c_back}')
    c_naive = count_naive_recursive(prefix,secuence)
    print(f'Naive Recursive: {c_naive}')
    c_dyn = count_dynamic(prefix,secuence)
    print(f'Naive Recursive: {c_dyn}')
    print()
    ##assert c_back == c_naive == c_dyn
    assert c_naive == c_dyn
    TEST+=1

TEST NO. 1
PREFIX: TDUATUDCAUTAAURAAAUGRGATC 
SECUENCE: DUATUDCAUTAAURAAAUGRGATCGTAATUDCAUTAAURAAAUGRGATCUTDATUDCAUTAAURAAAUGRGATCUDTDDAAAUGRGATCRTDUATUDCAUTAAUATUDCAUTAAURAAAUGRGATCUUTDTUDCAUTAAURAAAUGRGATCATDUCDUATUDCAUTAAURAAAUGRGATCATATUDCAUTAAURAAAUGRGATCUTDATTUDCAUTAAURAAAUGRGATCATDUUATUDCAUTAAURAAAUGRGATCUTDTUDCAUTAAURAAAUGRGATCADTDUDUATUDCAUTAAURAAAUGRGATCTGDUATUDCAUTAAURAAAUGRGATCDTAAAUGRGATCRTDUATUDCAUTAAUATUDCAUTAAURAAAUGRGATCTATDURGATCGDTDUATUDCAUTAAURAAAUDUATUDCAUTAAURAAAUGRGATCTUGATUDCAUTAAURAAAUGRGATCUTDCAAAUGRGATCRTDUATUDCAUTAAUATUDCAUTAAURAAAUGRGATCGUTDATUDCAUTAAURAAAUGRGATCUUTDDTUDCAUTAAURAAAUGRGATCATDUTUDCAUTAAURAAAUGRGATCACTDUAAAUGRGATCGRTDUATUDCAUTAAUTUDCAUTAAURAAAUGRGATCATDUTAAAUGRGATCRUTDUATUDCAUTAAUDUATUDCAUTAAURAAAUGRGATCDTTUDCAUTAAURAAAUGRGATCAUTDUAAAUGRGATCRRTDUATUDCAUTAAUTUDCAUTAAURAAAUGRGATCATDUTRTUDCAUTAAURAAAUGRGATCATDUTUDCAUTAAURAAAUGRGATCATDUGUATUDCAUTAAURAAAUGRGATCADTTUDCAUTAAURAAAUGRGATCARTDURGATCGRTDUATUDCAUTAAURAAAUTUDCAUTAAURAAAUGRGATCATDUTTUDCAUTA